In [1]:
# Primeiro, realizamos a instalação do mlflow e autoML do sklearn

#!pip install auto-sklearn

!pip install mlflow

!pip install tqdm 

!pip install pprint 

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [2]:
import mlflow
import requests

url = "http://checkip.amazonaws.com"
try:
    response = requests.get(url)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    IP = response.text.replace('\n', '')
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

uri_mlflow = "http://"+IP+":8089"
print(uri_mlflow)

mlflow.set_tracking_uri(uri_mlflow)

http://44.220.243.252:8089


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# LE O DATASET COMO UM PANDAS DATAFRAME.
df_data = pd.read_csv('curso_dw.csv')

# Definição das colunas que serão features (nota-se que a coluna NOME não está presente)
features = [
    #"MATRICULA",
    "REPROVACOES_MAT_1", 'REPROVACOES_MAT_2', "REPROVACOES_MAT_3", "REPROVACOES_MAT_4",
    "NOTA_MAT_1", "NOTA_MAT_2", "NOTA_MAT_3", "NOTA_MAT_4",
    "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
    #"INGLES_DESC", # usando todas as colunas
    #"CURSOU_MAT1_DESC", "CURSOU_MAT2_DESC", "CURSOU_MAT3_DESC", "CURSOU_MAT4_DESC",
]

# Definição da variável-alvo
target = ["PERFIL"]

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df_data[features]
y = df_data[target]

# Separação dos dados em um conjunto de treino e um conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=337)

In [4]:
import autosklearn.classification
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import threading
import time


# Parâmetros do AutoML
total_time = 300  # Tempo total (em segundos)
per_model_time = 30  # Tempo máximo por modelo

# Criando uma barra de progresso em thread separada
def show_progress_bar(duration):
    for _ in tqdm(range(duration), desc="AutoML training progress", ncols=100):
        time.sleep(1)

# Iniciando barra de progresso em segundo plano
progress_thread = threading.Thread(target=show_progress_bar, args=(total_time,))
progress_thread.start()

# Treinamento com auto-sklearn
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=total_time,
    per_run_time_limit=per_model_time,
    tmp_folder='/tmp/autosklearn_tmp',
    #output_folder='/tmp/autosklearn_out',
    disable_evaluator_output=False,
    n_jobs=-1
)

automl.fit(X_train, y_train)

# Espera barra terminar se for mais rápida
progress_thread.join()

# Resultados
print("\Ranking dos modelos testados: \n")
print(automl.leaderboard())

y_pred = automl.predict(X_test)
print(f"\Acurácia: {accuracy_score(y_test, y_pred):.4f}")


AutoML training progress: 100%|███████████████████████████████████| 300/300 [05:01<00:00,  1.01s/it]


\Ranking dos modelos testados: 

          rank  ensemble_weight               type      cost   duration
model_id                                                               
5            1             0.24      random_forest  0.193344   8.359419
9            2             0.18  gradient_boosting  0.194219   9.839831
26           3             0.02                mlp  0.200350  21.292783
4            4             0.02  gradient_boosting  0.218524   7.771238
21           5             0.02      liblinear_svc  0.219838   2.675278
33           6             0.02                mlp  0.220276   7.070588
16           7             0.04                lda  0.228596  14.865296
23           8             0.04      random_forest  0.237355  22.436863
24           9             0.38                lda  0.238012   2.294205
25          10             0.04                lda  0.241077   1.628145
\Acurácia: 0.8102


In [5]:
import pprint

pp = pprint.PrettyPrinter(indent=4)

# Resultados
print("\nModelos testados: \n")

pp.pprint(automl.show_models())


Modelos testados: 

{   4: {   'balancing': Balancing(random_state=1),
           'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7ff6e297a5e0>,
           'cost': 0.21852419531421063,
           'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7ff6f924a4c0>,
           'ensemble_weight': 0.02,
           'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7ff6e297a430>,
           'model_id': 4,
           'rank': 1,
           'sklearn_classifier': HistGradientBoostingClassifier(early_stopping=True,
                               l2_regularization=1.3153561911035717e-10,
                               learning_rate=0.025345400213312417, max_iter=256,
                               max_leaf_nodes=16, min_samples_leaf=27,
                               n_iter_no_change=5, random_state=1,
                               va

In [ ]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder


# Sample DataFrame : Exemplo com 3 formas geometricas com diferentes tamanhos e cores
data = { 'Shape': ['Circle', 'Square', 'Triangle', 'Circle'],
        'Color': ['Red', 'Blue', 'Green', 'Red'], 
        'Size': ['Small', 'Medium', 'Large', 'Small']}

df = pd.DataFrame(data)

print("Sample DataFrame:\n", df)

# Using LabelEncoder (sklearn)
df_le = pd.DataFrame()
le = LabelEncoder()
for col in df.columns:
    df_le[col + '_le'] = le.fit_transform(df[col])

print("\nLabelEncoder DataFrame:\n", df_le)


# Using get_dummies to encode multiple columns
df_encoded_multiple = pd.get_dummies(df, columns=['Color', 'Size', 'Shape'])
print("\nPandas get_dummies Encoded colunas:\n", df_encoded_multiple)

# Dropping the first dummy column to avoid multicollinearity
df_encoded_drop_first = pd.get_dummies(df, columns=['Color'], drop_first=True)
print("\nEncoded with drop_first column to avoid multicollinearity:\n", df_encoded_drop_first)


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# LE O DATASET COMO UM PANDAS DATAFRAME.
df_numeros = pd.read_csv('curso_dw.csv')

le = LabelEncoder()

df_numeros['INGLES_DESC'] = le.fit_transform(df_numeros['INGLES_DESC'])
df_numeros['CURSOU_MAT1_DESC'] = le.fit_transform(df_numeros['CURSOU_MAT1_DESC'])
df_numeros['CURSOU_MAT2_DESC'] = le.fit_transform(df_numeros['CURSOU_MAT2_DESC'])
df_numeros['CURSOU_MAT3_DESC'] = le.fit_transform(df_numeros['CURSOU_MAT3_DESC'])
df_numeros['CURSOU_MAT4_DESC'] = le.fit_transform(df_numeros['CURSOU_MAT4_DESC'])
df_numeros['PERFIL'] = le.fit_transform(df_numeros['PERFIL'])


print("\nLabelEncoder DataFrame: \n\n", df_numeros.head())

# Definição das colunas que serão features (nota-se que a coluna NOME não está presente)
features = [
    #"MATRICULA",
    "REPROVACOES_MAT_1", 'REPROVACOES_MAT_2', "REPROVACOES_MAT_3", "REPROVACOES_MAT_4",
    "NOTA_MAT_1", "NOTA_MAT_2", "NOTA_MAT_3", "NOTA_MAT_4",
    "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
    "INGLES_DESC", # usando todas as colunas
    "CURSOU_MAT1_DESC", "CURSOU_MAT2_DESC", "CURSOU_MAT3_DESC", "CURSOU_MAT4_DESC",
]

# Definição da variável-alvo
target = ["PERFIL"]

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df_numeros[features]
y = df_numeros[target]

# Separação dos dados em um conjunto de treino e um conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=337)


LabelEncoder DataFrame: 

    MATRICULA                       NOME  REPROVACOES_MAT_1  REPROVACOES_MAT_2  \
0     502375          Márcia Illiglener                  0                  0   
1     397093   Jason Jytereoman Izoimum                  0                  0   
2     915288  Bartolomeu Inácio da Gama                  0                  0   
3     192652            Fernanda Guedes                  1                  3   
4     949491     Alessandre Borba Gomes                  1                  3   

   REPROVACOES_MAT_3  REPROVACOES_MAT_4  NOTA_MAT_1  NOTA_MAT_2  NOTA_MAT_3  \
0                  0                  0         6.2         5.8         4.6   
1                  0                  0         6.0         6.2         5.2   
2                  0                  0         7.3         6.7         7.1   
3                  1                  1         0.0         0.0         0.0   
4                  1                  1         0.0         0.0         0.0   

   NOTA_MA

In [8]:
from sklearn import datasets

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#import autosklearn.classification
import autosklearn.regression


# Parâmetros do AutoML
total_time = 300  # Tempo total (em segundos)
per_model_time = 30  # Tempo máximo por modelo

print("Criando os modelos com AUTOML")
#cls = autosklearn.classification.AutoSklearnClassifier()
reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=total_time,
    per_run_time_limit=per_model_time,
    tmp_folder='/tmp/autosklearn_tmp',
    #output_folder='/tmp/autosklearn_out',
    disable_evaluator_output=False,
    n_jobs=-1
)

# Iniciando barra de progresso em segundo plano
progress_thread_reg = threading.Thread(target=show_progress_bar, args=(total_time,))
progress_thread_reg.start()

print("Treinando os modelos com AUTOML")
#cls.fit(X_train, y_train)
reg.fit(X_train, y_train)

print("Verificando resultados dos modelos com AUTOML")
#classification_predictions = cls.predict(X_test)
regression_predictions = reg.predict(X_test)

# Espera barra terminar se for mais rápida
progress_thread_reg.join()

# Resultados
print("\Ranking dos modelos testados: \n")
print(automl.leaderboard())

y_pred = automl.predict(X_test)
print(f"\Acurácia: {accuracy_score(y_test, y_pred):.4f}")

print("")
#print("Acurácia usando CLASSIFICACAO com AUTOML: {}%".format(100*round(accuracy_score(y_test, classification_predictions), 2)))
print("Acurácia usando REGRESSAO com AUTOML: {}%".format(100*round(accuracy_score(y_test, regression_predictions), 2)))



Criando os modelos com AUTOML
Treinando os modelos com AUTOML


AutoML training progress:  98%|██████████████████████████████████▎| 294/300 [04:55<00:06,  1.00s/it]

Verificando resultados dos modelos com AUTOML


AutoML training progress: 100%|███████████████████████████████████| 300/300 [05:01<00:00,  1.01s/it]


\Ranking dos modelos testados: 

          rank  ensemble_weight               type      cost   duration
model_id                                                               
5            1             0.24      random_forest  0.193344   8.359419
9            2             0.18  gradient_boosting  0.194219   9.839831
26           3             0.02                mlp  0.200350  21.292783
4            4             0.02  gradient_boosting  0.218524   7.771238
21           5             0.02      liblinear_svc  0.219838   2.675278
33           6             0.02                mlp  0.220276   7.070588
16           7             0.04                lda  0.228596  14.865296
23           8             0.04      random_forest  0.237355  22.436863
24           9             0.38                lda  0.238012   2.294205
25          10             0.04                lda  0.241077   1.628145


ValueError: X has 17 features, but ColumnTransformer is expecting 12 features as input.

In [ ]:
# Create a new MLflow Experiment
mlflow.set_experiment("MLflow AUTOML CLASSIFICACAO x REGRESSAO")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    #mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy_score(y_test, regression_predictions)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "AUTOML CLASSIFICACAO x REGRESSAO")

    # Infer the model signature
    signature = infer_signature(X_train, reg.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=reg,
        artifact_path="curso_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="automl-curso-perfil-alunos",
    )

In [ ]:
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

#iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=features)
result["actual_class"] = y_test
result["predicted_class"] = predictions

#result[:4]